In [1]:
import numpy as np
import pandas as pd
import pickle
import pprint 

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack

class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

train = pickle.load(open('train.p','rb')).sample(1000)
test = pickle.load(open('test.p','rb')).sample(1000)

train_text = train['comment_text']
test_text = test
all_text = pd.concat([train_text, test_text])

word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    ngram_range=(1, 4),
    max_features=400)
word_vectorizer.fit(all_text)
train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)
print('word TFIDF')

char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    ngram_range=(1, 4),
    max_features=400)
char_vectorizer.fit(all_text)
train_char_features = char_vectorizer.transform(train_text)
test_char_features = char_vectorizer.transform(test_text)
print('char TFIDF')

train_features = hstack([train_char_features, train_word_features])
test_features = hstack([test_char_features, test_word_features])

submission = pd.DataFrame.from_dict({'id': test.index.tolist()})
for class_name in class_names:
    train_target = train[class_name]
    print("training on ",class_name)
    clf1 = LogisticRegression(solver='sag')
    clf2 = RandomForestClassifier(n_estimators=100, max_depth=20, max_features=5000,n_jobs=-1)
    classifier = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2)], voting='soft', n_jobs=-1)

    classifier.fit(train_features, train_target)
    print("classification on ",class_name)
    submission[class_name] = classifier.predict_proba(test_features)[:, 1]
print('##################################')

submission.to_csv('Voting_lr_rndmfrst_char_word.csv', index=False)
print("this is a soft voting between LogisticRegression(solver='sag') and RandomForestClassifier(n_estimators=100, max_depth=20, max_features=5000,n_jobs=-1) with 40k word&char TFIDF features up to 4-grams together. The TFIDF for training model IS ON ALL TEST AND TRAIN TEXTS TOGETHER")

word TFIDF
char TFIDF
training on  toxic


NameError: name 'RandomForestClassifier' is not defined

In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.svm import SVC
import pickle
import re
import warnings
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, log_loss
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
warnings.filterwarnings("ignore")

In [3]:
submission = pd.DataFrame.from_dict({'id': test.index.tolist()})
for class_name in class_names:
    train_target = train[class_name]
    print("training on ",class_name)
    clf1 = LogisticRegression(solver='sag')
    clf2 = RandomForestClassifier(n_estimators=100, max_depth=20, max_features=5000,n_jobs=-1)
    classifier = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2)], voting='soft', n_jobs=-1)

    classifier.fit(train_features, train_target)
    print("classification on ",class_name)
    submission[class_name] = classifier.predict_proba(test_features)[:, 1]
print('##################################')

submission.to_csv('Voting_lr_rndmfrst_char_word.csv', index=False)
print("this is a soft voting between LogisticRegression(solver='sag') and RandomForestClassifier(n_estimators=100, max_depth=20, max_features=5000,n_jobs=-1) with 40k word&char TFIDF features up to 4-grams together. The TFIDF for training model IS ON ALL TEST AND TRAIN TEXTS TOGETHER")

training on  toxic
classification on  toxic
training on  severe_toxic
classification on  severe_toxic
training on  obscene
classification on  obscene
training on  threat
classification on  threat
training on  insult
classification on  insult
training on  identity_hate
classification on  identity_hate
##################################
this is a soft voting between LogisticRegression(solver='sag') and RandomForestClassifier(n_estimators=100, max_depth=20, max_features=5000,n_jobs=-1) with 40k word&char TFIDF features up to 4-grams together. The TFIDF for training model IS ON ALL TEST AND TRAIN TEXTS TOGETHER
